In [1]:
import calendar
import pandas as pd
import panel as pn

from datetime import date, timedelta
from sqlalchemy import create_engine

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 8, 21)

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trade name qty target active reason market xdate'.split()

In [3]:
pd.read_sql_query('SELECT * FROM orders ORDER BY name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,17,S,ASP,10000,3.82,0,CP1S,SET,2022-08-24
1,14,S,BCH,3000,21.00,0,CP1S,SET100,2022-08-31
2,2,B,BCP,3000,29.00,0,XXX,SET100,2022-02-02
3,3,B,BLA,4000,36.00,0,XXX,SET50,2022-08-23
4,1,B,CPNREIT,5000,19.00,0,RD02pct,SET,2022-08-17
5,4,B,CRC,3000,35.00,0,3L,SET50,2022-02-02
6,5,B,GVREIT,5000,9.00,0,XXX,SET,2022-08-26
7,6,B,IVL,3000,42.00,0,13WL,SET50,2022-08-30
8,15,S,KCE,1000,61.50,0,CP1S,SET50,2022-08-22
9,7,B,KSL,30000,3.30,0,6L,SET,2022-02-02


### After call Ord-Log.bat which calculates spreads between target price and SET price.

In [4]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name
#col_names   = ['trade','name','spreads','reason','market','qty','target_price','set_price','chg_amt','chg_pct','active']
df_log = pd.read_csv(input_file, sep=',',  index_col=None)
df_log

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate
0,B,BCP,-17,XXX,SET100,3000,29.00,33.25,0.50,+1.53%,0,2022-02-02
1,B,BLA,-2,XXX,SET50,4000,36.00,36.50,0.75,+2.10%,0,2022-08-23
2,B,CPNREIT,-2,RD02pct,SET,5000,19.00,19.20,-0.10,-0.52%,0,2022-08-17
3,B,CRC,-19,3L,SET50,3000,35.00,39.75,0.00,0.00%,0,2022-02-02
4,B,GVREIT,-3,XXX,SET,5000,9.00,9.15,-0.25,-2.66%,0,2022-08-26
5,B,IVL,-7,13WL,SET50,3000,42.00,43.75,-0.75,-1.69%,0,2022-08-30
6,B,KSL,-10,6L,SET,30000,3.30,3.50,-0.04,-1.13%,0,2022-02-02
7,B,PTT,-12,XXX,SET50,4500,34.00,37.00,-0.25,-0.67%,0,2022-03-03
8,B,PTTEP,-15,XXX,SET50,600,150.00,157.50,2.50,+1.61%,0,2022-08-15
9,B,STARK,-5,XXX,SET100,25000,4.00,4.10,-0.06,-1.44%,0,2022-02-02


In [5]:
def calc(vals):
    spreads = vals
    if abs(spreads) < 10:
        return 1
    else:
        return 0

In [6]:
df_log['active'] = df_log['spd'].apply(calc)

In [7]:
df_log.groupby('active')['trade'].count()

active
0    12
1     8
Name: trade, dtype: int64

In [8]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

20

In [9]:
final = df_log[colt]
final

,trade,name,qty,target,active,reason,market,xdate
0,B,BCP,3000,29.00,0,XXX,SET100,2022-02-02
1,B,BLA,4000,36.00,1,XXX,SET50,2022-08-23
2,B,CPNREIT,5000,19.00,1,RD02pct,SET,2022-08-17
3,B,CRC,3000,35.00,0,3L,SET50,2022-02-02
4,B,GVREIT,5000,9.00,1,XXX,SET,2022-08-26
5,B,IVL,3000,42.00,1,13WL,SET50,2022-08-30
6,B,KSL,30000,3.30,0,6L,SET,2022-02-02
7,B,PTT,4500,34.00,0,XXX,SET50,2022-03-03
8,B,PTTEP,600,150.00,0,XXX,SET50,2022-08-15
9,B,STARK,25000,4.00,1,XXX,SET100,2022-02-02


In [10]:
rcds = final.values.tolist()
rcds

[['B', 'BCP', 3000, 29.0, 0, 'XXX', 'SET100', '2022-02-02'],
 ['B', 'BLA', 4000, 36.0, 1, 'XXX', 'SET50', '2022-08-23'],
 ['B', 'CPNREIT', 5000, 19.0, 1, 'RD02pct', 'SET', '2022-08-17'],
 ['B', 'CRC', 3000, 35.0, 0, '3L', 'SET50', '2022-02-02'],
 ['B', 'GVREIT', 5000, 9.0, 1, 'XXX', 'SET', '2022-08-26'],
 ['B', 'IVL', 3000, 42.0, 1, '13WL', 'SET50', '2022-08-30'],
 ['B', 'KSL', 30000, 3.3, 0, '6L', 'SET', '2022-02-02'],
 ['B', 'PTT', 4500, 34.0, 0, 'XXX', 'SET50', '2022-03-03'],
 ['B', 'PTTEP', 600, 150.0, 0, 'XXX', 'SET50', '2022-08-15'],
 ['B', 'STARK', 25000, 4.0, 1, 'XXX', 'SET100', '2022-02-02'],
 ['S', 'ASP', 10000, 3.82, 0, 'CP1S', 'SET', '2022-08-24'],
 ['S', 'BCH', 3000, 21.0, 0, 'CP1S', 'SET100', '2022-08-31'],
 ['S', 'KCE', 1000, 61.5, 0, 'CP1S', 'SET50', '2022-08-22'],
 ['S', 'MCS', 25000, 12.9, 0, 'CP1S', 'SET', '2022-09-03'],
 ['S', 'PTTGC', 3000, 51.5, 0, 'CP1S', 'SET50', '2022-09-03'],
 ['S', 'STA', 2500, 26.0, 0, 'CP1S', 'SET100', '2022-08-23'],
 ['S', 'TMT', 12000, 9.

In [11]:
for rcd in rcds:
    print(rcd)

['B', 'BCP', 3000, 29.0, 0, 'XXX', 'SET100', '2022-02-02']
['B', 'BLA', 4000, 36.0, 1, 'XXX', 'SET50', '2022-08-23']
['B', 'CPNREIT', 5000, 19.0, 1, 'RD02pct', 'SET', '2022-08-17']
['B', 'CRC', 3000, 35.0, 0, '3L', 'SET50', '2022-02-02']
['B', 'GVREIT', 5000, 9.0, 1, 'XXX', 'SET', '2022-08-26']
['B', 'IVL', 3000, 42.0, 1, '13WL', 'SET50', '2022-08-30']
['B', 'KSL', 30000, 3.3, 0, '6L', 'SET', '2022-02-02']
['B', 'PTT', 4500, 34.0, 0, 'XXX', 'SET50', '2022-03-03']
['B', 'PTTEP', 600, 150.0, 0, 'XXX', 'SET50', '2022-08-15']
['B', 'STARK', 25000, 4.0, 1, 'XXX', 'SET100', '2022-02-02']
['S', 'ASP', 10000, 3.82, 0, 'CP1S', 'SET', '2022-08-24']
['S', 'BCH', 3000, 21.0, 0, 'CP1S', 'SET100', '2022-08-31']
['S', 'KCE', 1000, 61.5, 0, 'CP1S', 'SET50', '2022-08-22']
['S', 'MCS', 25000, 12.9, 0, 'CP1S', 'SET', '2022-09-03']
['S', 'PTTGC', 3000, 51.5, 0, 'CP1S', 'SET50', '2022-09-03']
['S', 'STA', 2500, 26.0, 0, 'CP1S', 'SET100', '2022-08-23']
['S', 'TMT', 12000, 9.05, 0, 'CP1S', 'SET', '2022-08-24

In [12]:
sql = """
INSERT INTO orders (trade, name, qty, price, active, reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active, reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)



In [13]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [14]:
sql = '''
SELECT *
FROM orders
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)
df_tab = pn.widgets.Tabulator(df_orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=    id trade  ..., width=800)

In [15]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)